# Generalized Advantage Estimation

## 목적

- 현재 policy gradient method의 목적은 누적 discount reward를 optimization하는 것이다. 하지만 학습할 때 많은 양의 sample이 필요하고 sample data가 nonstationarity임에도 stable한 학습이 어렵다.
- 따라서, \\(TD(\lambda)\\)와 같은 advantage function의 exponentially-weighted estimator를 사용하여 policy gradient estimate의 variance를 줄인다. variance를 줄이게 되면 stable하게 학습을 할 수 있다.
- 또한, policy와 value function에 대한 Trust Region optimization(앞선 TRPO에서 소개되었다.) 사용

## 요약

- 따라서, advantage function의 good estimate을 얻는 variance reduction에 대해서 알아보자.
  - 위 문제를 해결하기 위해서, Generalized advantage estimator를 사용했고, 이것은 bias-variance tradeoff를 조절하는 두개의 parameter \\(\gamma, \lambda\\)가 있다.
  - 그리고, Trust region policy optimization과 value function을 optimize하는 trust region 알고리즘의 아이디어를 어떻게 합치는지 증명하였다.

- REINFORCE, expected total returns의 gradient에 대한 unbiased estimate을 REINFORCE라 한다. 하지만 하나의 에피소드가 끝날 때 update를 진행하는 방식이므로 높은 variance를 가지게 된다.
- Actor-critic, 하나의 value function을 사용한다. 그리고 bias하고 낮은 variance를 가진다. 만약 높은 variance를 가진다면 sampling을 더 하면 되는데 bias한 경우는 치명적이다. bias한 경우는 수렴이 되지 않거나 local optimum이 아닌 poor solution에 수렴하도록 만든다.

## 준비

- 일단 policy optimization의 undicounted formulation을 가정한다. (원래는 discount를 붙여주지만 여기선 붙이지 않는 것을 가정한다.)
- initial state \\(s_0\\)는 distribution \\(\rho_0\\)로 부터 sampling된 것이다.
- 하나의 trajectory(\\(s_0, a_0, s_1, a_1, ...)\\))는 terminal state에 도달할 때 까지 policy \\(a_t \sim \pi(a_t|s_t)\\)에 따라서 action을 sampling하고, dynamics \\(s_{t+1} \sim P(s_{t + 1}|s_t, a_t)\\)에 따라서 state를 sampling함으로써 생성된다.
- reward, \\(r_t = r(s_t, a_t, s_{t+1})\\)은 매 time step마다 받아진다.
- 우리의 목표는 모든 policy에 대해서 finite하다고 가정됨으로서 expected total reward \\(\sum^\infty_{t=0} r_t\\)를 maximize하는 것이다.


- 앞서 가정했듯이 undiscounted formulation이고, 따라서 \\(\gamma\\)를 discount의 parameter로 사용하는 것이 아니라 bias-variance tradeoff를 조절하는 parameter로 사용한다.

- policy gradient method는 gradient \\(g := \triangledown_\theta \mathbb{E}[\sum^\infty_{t=0} r_t] \\)를 반복적으로 estimate함으로써 expected total reward를 maximize하는 것인데, policy gradient에는 여러가지 표현들이 있다.

$$ g = \mathbb{E}[\sum^\infty_{t=0} \Phi_t \triangledown_\theta log \pi_\theta(a_t|s_t)] $$
- 위 식에서 \\(\Phi_t\\)는 아래식 중 하나가 될 수 있다.
  1. \\(\sum^\infty_{t=0} r_t\\), trajectory의 total reward
  2. \\(\sum^\infty_{t'=t} r'_t\\), action \\(a_t\\)후의 reward
  3. \\(\sum^\infty_{t'=t} r'_t - b(s_t)\\), 위 2번 식의 baselined된 reward
  4. \\(Q^\pi(s_t, a_t)\\), state-action value function
  5. \\(A^\pi(s_t, a_t)\\), advantage function, advantage를 빼줘서 높은 variance를 보정한다.
  6. \\(r_t + V^\pi(s_{t+1}) - V^\pi(s_t)\\), TD error
- 위 식들중 4, 5, 6은 아래 수식을 사용한다.
  - \\(V^\pi(s_t) := \mathbb{E}_{s_{t+1}:\infty, a_t:\infty}[\sum^\infty_{l=0} r_{t+1}]\\)
  - \\(Q^\pi(s_t, a_t) := \mathbb{E}_{s_{t+1}:\infty, a_{t+1}:\infty}[\sum^\infty_{l=0} r_{t+l}]\\)
  - \\(A^pi(s_t, a_t) := Q^\pi(s_t, a_t) - V^\pi(s_t)\\)
- notation
  - \\(a:b\\)의 의미는, \\((a, a + 1, ... , b)\\)이다.

## parameter \\(\gamma\\)

- parameter \\(\gamma\\)는 bias하면서 동시에 reward를 downweighting함으로써 variance를 줄인다.
- 위 말은, MDP에서 discounted formulation에 사용되는 discount factor와 같지만, 논문의 의도는 undiscounted MDP에서 variance reduction parameter로 생각한다.
  - 하지만 결과는 같다. 그렇지만 의도는 다르다.

- 따라서, policy gradient에서 사용되는 discounted approximation은 최종적으로 아래의 식으로 나타낼 수 있다.
  - \\(g^{\gamma} := \mathbb{E}_{s_{0:\infty}, a_{0:\infty}}[\sum^\infty_{t=0} A^{\pi,\gamma}(s_t, a_t) \triangledown_\theta log_{\pi_\theta}(a_t|s_t)]\\)

### \\(\gamma\\)-just estimator

- \\(\gamma\\)-just estimator는 \\(g^\gamma\\)를 estimate하기 위해 위의 수식에서 \\(A^{\pi, \gamma}\\)를 \\(\gamma\\)-just estimator로 사용할 때, bias하지 않은 estimator라 하자. 그리고 이 \\(\gamma\\)-just advantage estimator를 \\(\hat{A}_t(s_{0:\infty}, a_{0:\infty})\\)라 하고, 전체의 trajectory에 대한 하나의 function이라 하자.
- \\(A^\gamma\\)를 \\(A^{\pi, \gamma}\\)로 사용함으로써 이미 bias하다고 말했지만, 이 논문에서는 \\(g^\gamma\\)에 대해 unbiased estimate을 얻고 싶은 것이다. 하지만 undiscounted MDP의 policy gradient에 대해서는 당현히 \\(\gamma\\)를 사용하기 때문에 biased estimate을 얻는다.
  - 따라서, 일단 최종 목적은 \\(\gamma\\)를 사용할 때 어떻게 unbiased estimate을 얻을지 생각해보자.
  - 위 식에서 \\(A^{\pi, \gamma}\\)를 \\(\gamma\\)-just estimator로 바꿔줌으로써 unbiased estimate을 하고싶은 것이다.

### 정의

- \\(\mathbb{E}_{s_{0:\infty}, a_{0:\infty}}[\hat{A}_t(s_{0:\infty}, a_{0:\infty}) \triangledown_\theta log \pi_\theta (a_t|s_t)] = \mathbb{E}_{s_{0:\infty}, a_{0:\infty}}[A^{\pi, \gamma}(s_t, a_t) \triangledown_\theta log \pi_\theta(a_t|s_t)]\\)
  - 위 식을 만족하면, estimator \\(\hat{A}_t\\)는 \\(\gamma\\)-just이다.

- 만약 모든 t에 대해서 \\(\hat{A}_t\\)이 \\(\gamma\\)-just라면, 아래 식으로 표현 가능하다.
  - \\(\mathbb{E}_{s_{0:\infty}, a_{0:\infty}}[\hat{A}_t(s_{0:\infty}, a_{0:\infty}) \triangledown_\theta log \pi_\theta (a_t|s_t)] = g^\gamma\\)
  - 위 식이 unbiased estimate이다.

- \\(\gamma\\)-just인 \\(\hat{A}_t\\)에 대한 한가지 조건은 \\(\hat{A}_t\\)이 두가지 function \\(Q_t\\)와 \\(b_t\\)로 나뉠 수 있다는 것이다.
  - \\(Q_t\\)는 \\(\gamma\\)-discounted Q-function의 unbiased estimator이다.
  - \\(b_t\\)는 action \\(a_t\\)전에 sampling된 state and actions의 arbitrary function이다.

### Proposition

- 모든 \\(s_t, a_t\\)에 대해서,
$$ \mathbb{E}_{s_{t+1:\infty}, a_{t+1:\infty}|s_t, a_t}[Q_t(s_{t:\infty}, a_{t:\infty})] = Q^{\pi, \gamma}(s_t, a_t) $$
- 위 식에 의해 \\(\hat{A}_t\\)가 아래와 같다 가정하자. 이때 \\(\hat{A}_t\\)는 \\(\gamma\\)-just이다.
$$ \hat{A}_{s_{0:\infty}, a_{0:\infty}} = Q_t(s_{0:\infty}, a_{0:\infty}) - b_t(s_{0:t}, a_{0:t-1}) $$

### Advantage function estimation

- discounted advantage function \\(A^{\pi, \gamma}(s_t, a_t)\\)의 accurate estimate \\(\hat{A}_t\\)에 대해 알아보자.
$$ \hat{g} = \frac{1}{N} \sum^N_{n=1} \sum^\infty_{t=0} \hat{A}^n_t \triangledown_\theta log \pi_\theta(a^n_t|s^n_t) $$
- 위 식에서 n은 에피소드의 batch에 대한 index이다.

- \\(V\\)를 approximate value function라 하고, \\(\delta^V_t = r_t \gamma V(s_{t+1}) - V(s_t) \\)라 하자.
  - correct value function이 \\(V = V^{\pi, \gamma})\\)가 있다고 하면, 이것은 \\(\gamma \\)-just advantage estimator이다.
  - 실제로, \\(A^{\pi, \gamma}\\)의 unbiased estimator는 아래와 같다.
    $$ \begin{split} \mathbb{E}_{s_{t+1}}[\delta^{V^{\pi, \gamma}_t}] &= \mathbb{E}_{s_{t+1}}[r_t + \gamma V^{\pi, \gamma}(s_{t + 1}) - V^{\pi, \gamma}(s_t)] \\ &= \mathbb{E}_{s_{t+1}}[Q^{\pi, \gamma}(s_t, a_t) - V^{\pi, \gamma}(s_t)] \\ &= A^{\pi, \gamma}(s_t, a_t) \end{split} $$
  - 하지만 위 estimator는 \\(V = V^{\pi, \gamma}\\)에 대한 \\(\gamma\\)-just이다. 다른 경우라면 이것은 biased policy gradient estimate일 것이다. 하지만 우리의 목표는 V에 대해서만 unbiased가 아니라 advantage function에 대해서 일반화된 unbiased estimator가 필요하다.

- 따라서, \\(\delta\\)에 대해 \\(k\\)의 합으로 생각해보면, \\(\hat{A}^{(k)}_t\\)라 하고 아래 식으로 표현 가능하다.
  $$ \hat{A}^{(1)}_t := \delta^V_t \qquad = -V(s_t) + r_t + \gamma V(s_{t+1}) $$ <br><br><br>
  $$ \hat{A}^{(k)}_t := \sum^{k - 1}_{l = 0} \gamma^l \delta^V_{t+l} = -V(s_t) + r_t + \gamma r_{t+1} + ... + \gamma^{k - 1} r_{t+k-1} + \gamma^k V(s_{t+k}) $$
  - \\(\hat{A}^{(k)}_t\\)는 return의 k-step estimate와 연관지을 수 있고, \\(\delta^V_t = \hat{A}^{(1)}_t\\)의 case와 유사하게도 \\(\hat{A}^{(k)}_t\\)를 advantage function의 estimate로 생각할 수 있다.

- 여기서, \\(k -> \infty\\)로 생각하면 bias가 일반적으로 점점 더 작아진다. 왜냐하면 \\(\gamma^k V(s_{t+k})\\)가 점점 많이 discounted되서 \\(-V(s_t)\\)가 bias에 영향을 미치지 못하기 때문이다. 그래서 \\(k -> \infty\\)를 취하면 아래와 같은 식이 나온다.
  $$ \hat{A}^{(\infty)}_t = \sum^\infty_{l=0} \gamma^l \delta^V_{t+l} = -V(s_t) + \sum^\infty_{l=0} \gamma^l r_{t+l} $$
  - 우변의 수식과 같이 경험적 return에서 value function baseline을 뺀 것으로 나타낼 수 있다.

### Generlized advantage estimator GAE(\\(\gamma, \lambda\\))

- k-step estimators의 exponentially-weighted average로 나타낼 수 있다.
  $$ \begin{split} \hat{A}^{GAE(\gamma, \lambda)}_t &:= (1-\lambda)(\hat{A}^{(1)}_t + \lambda \hat{A}^{(2)}_t + \lambda^2 \hat{A}^{(3)}_t + ...) \\
  &= (1-\lambda)(\delta^V_t + \lambda(\delta^V_t + \gamma \delta^V_{t+1}) + \lambda^2 (\delta^V_t + \gamma \delta^V_{t+1} + \gamma^2 \delta^V_{t+2}) + ... ) \\
  &= (1-\lambda)(\delta^V_t(1 + \lambda + \lambda^2 + ...) + \gamma \delta^V_{t+1}(\lambda + \lambda^2 + \lambda^3 + ...) + \gamma \delta^V_{t+2}(\lambda^2 + \lambda^3 + ...) + ... ) \\
  &= (1-\lambda)(\delta^V_t(\frac{1}{(1-\lambda)}) + \gamma \delta^V_{t+1}(\frac{\lambda}{(1-\lambda)}) + \gamma^2 \delta^V_{t+2}(\frac{\lambda^2}{(1-\lambda)}) + ...) \\
  &= \sum^\infty_{l=0} (\gamma \lambda)^l \delta^V_{t + l} \end{split} $$

- 위 식에서 \\(\lambda = 0, \lambda = 1\\)에 대해서는 특별한 case가 있다. 아래 수식과 같이 표현할 수 있다.
  $$ \begin{split} GAE(\gamma, 0) &: \hat{A}_t &:= \delta_t &= r_t + \gamma V(s_{t+1}) - V(s_t) \\
  GAE(\gamma, 1) &: \hat{A}_t &:= \sum^\infty_{l=0} \gamma^l \delta_{t+l} &= \sum^\infty_{l=0} \gamma^l r_{t + l} - V(s_t) \end{split} $$
  - \\(GAE(\gamma, 1)\\)은 V의 정확도와 관계없이 \\(\gamma\\)-just이다. 그러나 return의 sum때문에 high variance하다.
  - \\(GAE(\gamma, 0)\\)은 \\(V = V^{\pi, \gamma}\\)에 대해 \\(\gamma\\)-just이다. 그리고 bias하지만 일반적으로 낮은 variance를 가진다.

- parameter \\(\gamma, \lambda\\)를 가지고 있는 advantage estimator는 bias-variance tradeoff에 도움을 준다. 그러나 이 두 parameter는 다른 목적을 가지고 움직인다.
  - \\(\gamma\\)는 value function \\(V^{\pi, \gamma}\\)의 scale을 결정한다. 또한, \\(\gamma\\)는 \\(\lambda\\)에 의존하지 않는다.
  - \\(\lambda < 1\\)은 유일하게 value function이 부정확할 때 bias하다. 그리고 경험상, \\(\lambda\\)의 best value는 \\(gamma\\)의 best value 보다 훨씬 낮다. 왜냐하면 \\(\lambda\\)가 정확한 value function에 대해 \\(\gamma\\)보다 훨씬 덜 bias하기 때문이다.

### 최종

$$ g^\gamma \approx \mathbb{E}[\sum^\infty_{t=0} \triangledown_\theta log \pi_\theta(a_t|s_t) \hat{A}^{GAE(\gamma, \lambda)}_t] = \mathbb{E}[\sum^\infty_{t=0} \triangledown_\theta log \pi_\theta(a_t|s_t) \sum^\infty_{l=0} (\gamma \lambda)^l \delta^V_{t+1}] $$
- \\(\lambda = 1\\)일 때 동일해진다.

## Reward shaping

### 목적

- reward space가 sparse한 경우 reward가 너무 드문드문 나온다. 따라서, 이것을 꾸준히 reward를 받을 수 있도록 바꾼다.
- 즉, 돌고래가 점프하여 불구멍을 통과하는 환경을 예로 들때, 점프를 해서 불을 피해서 구덩이를 통과한 후 착지까지 해야 reward +1을 받게 되는 환경이기 때문에 reward를 받기 전 까지 잘하고 있는지 못하는지 판단을 할 수 없다.
- 따라서, reward를 꾸준히 받을 수 있도록 설계한다.

### 방법

- potential-based shaping function \\(\Phi\\)를 만들어서 더하고 빼준다. \\(\Phi\\)에는 state value function이 주로 들어가게 된다.

- transformed reward function \\(\tilde{r}\\),
  - \\(\tilde{r}(s, a, s') = r(s, a, s') + \gamma \Phi(s') - \Phi(s) \\)
    - \\(\Phi\\), \\(S->\mathbb{R}\\)을 state space에서의 arbitrary scalar-valued function을 나타낸다.
    - 형태가 TD error와 비슷하지만 의미와 의도가 다르다. reward shaping은 sparse reward 때문이고, gae는 variance reduction 때문이다.

### trajectory의 rewards의 discounted sum 표현

- 위 transformation은 discounted advantage function \\(A^{\pi, \gamma}\\)로 둘 수 있다.
  - \\(\sum^\infty_{l=0} \gamma^l \tilde{r}(s_{t+l}, a_t, s_{t+l+1}) = \sum^\infty_{l=0} \gamma^l r(s_{t+l}, a_{t+l}, s_{t+l+1}) - \Phi(s_t)\\)

- \\(\tilde{Q}^{\pi, \gamma}, \tilde{V}^{\pi, \gamma}, \tilde{A}^{\pi, \gamma}\\)를 transformed MDP의 value function과 advantage function이라 하면, 아래와 같은 수식이 유도된다.
  - \\(\tilde{Q}^{\pi, \gamma} = Q^{\pi, \gamma}(s, a) - \Phi(s) \\)
  - \\(\tilde{V}^{\pi, \gamma} = V^{\pi, \gamma}(s, a) - \Phi(s) \\)
  - \\(\tilde{A}^{\pi, \gamma} = (Q^{\pi, \gamma}(s, a) - \Phi(s)) - (V^{\pi, \gamma}(s, a) - \Phi(s)) = A^{\pi, \gamma}(s,a) \\)

- 그렇다면 reward shaping으로 어떻게 policy gradient estimate을 얻을 수 있나?
  - \\(0 \leq \lambda \leq 1\\)의 범위에 있는 steeper discount \\(\gamma \lambda\\)를 사용한다.
  - shaped reward \\(\tilde{r}\\)은 bellman residual term \\(\delta^V\\)와 동일하다.
  - 마지막으로, \\(\Phi = V\\)가 같다고 하면 아래와 같은 수식이 도출된다.
    - \\(\sum^\infty_{l=0} (\gamma \lambda)^l \tilde{r}(s_{t+l}, a_t, s_{t+l+1}) = \sum^\infty_{l=0} (\gamma \lambda)^l \delta^V_{t+l} = \hat{A}^{GAE(\gamma, \lambda)}_t \\)
  - shape rewards의 \\(\gamma \lambda\\)-discounted sum을 고려함으로써 GAE를 얻을 수 있다. \\(\lambda = 1\\)은 \\(g^\gamma\\)의 unbiased estimate이고, \\(\lambda < 1\\)은 biased estimate이다.

- shaping transformation과 parameters \\(\gamma, \lambda\\)의 결과를 보기 위해, response function \\(\mathcal{X}\\)를 이용하면 아래와 같은 수식이 도출된다.
  - \\(\mathcal{X}(l;s_t, a_t) = \mathbb{E}[r_{t+l}|s_t, a_t] - \mathbb[r_{t+l}|s_t] \\)
  - 추가로, \\(A^{\pi, \gamma}(s,a) = \sum^\infty_{l=0} \gamma^l \mathcal{X}(l;s, a)\\)이다.

- 따라서, discounted policy gradient estimator는 아래와 같이 사용가능하다.
  - \\( \triangledown_\theta log \pi_\theta(a_t|s_t) A^{\pi, \gamma}(s_t, a_t) = \triangledown_\theta log \pi_\theta(a_t|s_t) \sum^\infty_{l=0} \gamma^l \mathcal{X}(l;s, a) \\)

### Value function estimation

- TRPO, GAE 결합

- value function을 나타내기 위한 nonlinear function approximator를 사용할 때, 가장 간단한 방법은 nonlinear regression problem을 해결하는 것이다.
  - \\(minimize_\phi \sum^N_{n=1} {\parallel V_\phi(s_n) - \hat{V}_n \parallel}^2\\)
  - \\(\hat{V}_t = \sum^\infty_{l=0} \gamma^l r_{t+l} \\)은 rewards의 discounted sum이다. 그리고 n은 trajectories에서 모든 timesteps에 대한 index이다.

- value function optimize를 위해서 trust region method를 사용한다.
  - trust region을 함으로써 overfitting을 피한다.
  - trust region problem을 공식화하기 위해, \\( \sigma^2 = \frac{1}{N} \sum^N_{n=1} {\parallel V_{\phi_{old}}(s_n) - \hat{V}_n \parallel}^2\\)를 계산한다.
  - 따라서 constrained optimization problem은
    - \\(minimize_\phi \sum^N_{n=1} {\parallel V_\phi(s_n) - \hat{V}_n \parallel}^2\\)
    - \\(subject \;to \frac{1}{N} \sum^N_{n=1} \frac{{\parallel V_\phi(s_n) - V_{\pi_{old}}(s_n) \parallel}^2}{2 \sigma^2} \leq \epsilon \\)
  - 위 constraint는 이전 value function과 새로운 value function의 사이를 \\(\epsilon\\)보다 더 작게 하기 위해서 average KL-divergence를 constraint하는 것이다. 여기서 value function은 평균 \\(V_\phi(s)\\)와 분산 \\(\sigma^2\\)를 가진 Gaussian distribution을 parameterize하기 위해 사용된다.

- conjugate gradient algorithm을 사용하여 위 trust region problem의 approximation solution을 해결한다.
  - \\(minimize_\phi g^T(\phi - \phi_{old})\\)
  - \\(subject \;to \frac{1}{N} \sum^N_{n=1} (\phi - \phi_{old})^T H(\phi - \phi_{old}) \leq \epsilon \\)
  - 여기서 g는 objective의 gradient이고, \\(H = \frac{1}{N}\sum_n j_n j^T_n\\)이다. 여기서 \\(j_n = \bigtriangledown_\phi V_\phi(s_n)\\)이다.
  - H는 objective의 Hessian의 approximation이다. 그래서 H는 Fisher information matrix가 된다.
  - conjugate gradient algorithm을 사용하기 위해서 matrix-vector products \\(v -> Hv\\)를 사용할 때, step direction \\(s \approx -H^-1 g\\)를 계산한다.
  - 그후, \\(\frac{1}{2}(\alpha s)^T H(\alpha s) = \epsilon\\)에 따라 \\(s -> \alpha s\\)로 rescaling한다. 그리고 \\(\phi = \phi_{old} + \alpha s\\)를 해준다.

# 참조

[GAE 설명 블로그](http://dongminlee.tistory.com/12)

### 위 블로그를 참조 해서 작성하였고, 중간중간에 어려운 개념들 공부하면서 더 추가할 예정입니다. 감사합니다.